In [79]:
package Encoder{
  #The base encoder interface for the encoder-decoder architecture.
  use strict;
  use warnings;
  use Data::Dump qw(dump);
  use AI::MXNet qw(mx);
  use AI::MXNet::AutoGrad qw(autograd);
  use AI::MXNet::Base;
  use AI::MXNet::Gluon::Block;
  use base ("AI::MXNet::Gluon::Block");

  sub new{
    my $self = shift;
    $self->SUPER::new();
  }

  sub forward{
    my ($self, $X, %args) = (shift, @_);
    print "NotImplementedEncoder.\n";
  }
  
  1;
}

1

Warning: Subroutine new redefined at reply input line 12.

Subroutine forward redefined at reply input line 17.


In [80]:
package Decoder{
  #The base decoder interface for the encoder-decoder architecture.
  use strict;
  use warnings;
  use Data::Dump qw(dump);
  use base ("AI::MXNet::Gluon::Block");

  sub new{
    my $self = shift;
    $self->SUPER::new();
  }

  sub init_state{
    my ($self, $enc_outputs, %args) = (shift, @_);
    print "NotImplementedDecoder.\n";
  }
  
  sub forward{
    my ($self, $X, $state) = (shift, @_);
    print "NotImplementedDecoder.\n";
  }
  
  1;
}

1

Warning: Subroutine new redefined at reply input line 8.

Subroutine init_state redefined at reply input line 13.

Subroutine forward redefined at reply input line 18.


In [81]:
package EncoderDecoder{
  #The base class for the encoder-decoder architecture.
  use strict;
  use warnings;
  use Data::Dump qw(dump);
  use base ("AI::MXNet::Gluon::Block");
  
  sub new{
    my $class = shift;
    my $self = $class->SUPER::new();
    $self->{encoder} = shift;
    $self->register_child($self->{encoder});
    $self->{decoder} = shift;
    $self->register_child($self->{decoder});
    return bless($self, $class);
  }

  sub forward{
    my ($self, $enc_X, $dec_X, $args) = @_;
    my $enc_outputs = [$self->{encoder}->($enc_X, $args)];
    my $dec_state = $self->{decoder}->init_state($enc_outputs, $args);
    
    return $self->{decoder}->($dec_X, $dec_state);
  }
  
  1;
}

1

Warning: Subroutine new redefined at reply input line 8.

Subroutine forward redefined at reply input line 18.


In [82]:
#@save
package Seq2SeqEncoder{
  #The RNN encoder for sequence to sequence learning.
  use strict;
  use warnings;
  use Data::Dump qw(dump);
  use AI::MXNet::Gluon qw(gluon);
  use AI::MXNet::Gluon::NN qw(nn);
  use base ("Encoder");#"dl2::Encoder"
  
  sub new{
    my ($class, %args) = (shift, @_);
    my $self = $class->SUPER::new();

    # Embedding layer
    $self->{embedding} = gluon->nn->Embedding($args{vocab_size}, $args{embed_size});
    $self->register_child($self->{embedding});
    $self->{rnn} = gluon->rnn->GRU(hidden_size => $args{num_hiddens}, num_layers => $args{num_layers}, dropout => $args{dropout} // 0);
    $self->register_child($self->{rnn});
    return bless($self, $class);
  }
  
  sub forward{
    my ($self, $X) = (shift, @_);
    # The output `X` shape: (`batch_size`, `num_steps`, `embed_size`)
    $X = $self->{embedding}->forward($X);
    # In RNN models, the first axis corresponds to time steps
    $X = $X->swapaxes(0, 1);
    my $state = $self->{rnn}->begin_state(@{$X->shape}[1], ctx => mx->cpu(0));
    my $output = $self->{rnn}->forward($X, $state);
    ($output, $state) = ($output->[0], $output->[1]);
    # `output` shape: (`num_steps`, `batch_size`, `num_hiddens`)
    # `state[0]` shape: (`num_layers`, `batch_size`, `num_hiddens`)
    return ($output, $state);
  }
  
  1;
}

1

Warning: Subroutine new redefined at reply input line 11.

Subroutine forward redefined at reply input line 23.


In [123]:
use strict;
use warnings;
use Data::Dump qw(dump);
use List::Util qw( min max shuffle);
use numperl;
use extension;
use constant np => 'numperl';
use constant ex => 'extension';
use constant False => 'False';
use constant True => 'True';
use constant None => 'None';
use AI::MXNet qw(mx);
use AI::MXNet qw(nd);
use AI::MXNet::AutoGrad qw(autograd);
use d2l;

In [205]:
package AttentionDecoder{
  use strict;
  use warnings;
  use Data::Dump qw(dump);
  use AI::MXNet::Gluon qw(gluon);
  use AI::MXNet::Gluon::NN qw(nn);
  use base ("Decoder");#"dl2::Decoder"
  
    sub new{
    my $self = shift;
    $self->SUPER::new();
  }
    
    sub attention_weights1{
        my($self) =@_;
        print "NotImplementedError.\n";
    }

}

Warning: Subroutine new redefined at reply input line 9.

Subroutine attention_weights1 redefined at reply input line 14.


In [221]:
package AdditiveAttention{
  use strict;
  use warnings;
  use constant np => 'numperl';
  use Data::Dump qw(dump);
   use AI::MXNet::Gluon qw(gluon);
  use AI::MXNet::Gluon::NN qw(nn);
  use base ("AI::MXNet::Gluon::Block");
  
  sub new{
    my ($class, %args) = (shift, @_);
    my $self = $class->SUPER::new();
    $self->{'W_k'}= gluon->nn->Dense($args{'num_hiddens'}, use_bias=>0, flatten => 0);
    $self->register_child($self->{'W_k'});
    $self->{'W_q'}= gluon->nn->Dense($args{'num_hiddens'}, use_bias=>0, flatten => 0);
   $self->register_child($self->{'W_q'});
    $self->{'dropout'} = gluon->nn->Dropout($args{'dropout'});
    $self->register_child($self->{'dropout'});
    return bless($self, $class);
    }
    
    sub forward{
        my($self, $queries, $keys, $values, $valid_lens)=@_;
        $queries= $self->{'W_q'}($queries);
        $keys= $self->{'W_k'}->($keys);
        my $features = np->expand_dims($queries, axis=>2) + np->expand_dims($keys, axis=>1);
        $features = np->tanh($features);
        my $scores = np->squeeze($self->{'w_v'}($features), axis=> -1);
        $self->{'attention_weights'} = masked_softmax($scores, $valid_lens);
        return mx->nd->batch_dot($self->{'dropout'}($self->{attention_weights}), $values);
    }
}

Warning: Subroutine new redefined at reply input line 10.

Subroutine forward redefined at reply input line 22.


In [238]:
#my $queries = np->random_normal(loc=>0, scale=>1, size=>[2, 1, 20]);
#my $keys=  np->ones([2, 10, 2]);
# The two value matrices in the `values` minibatch are identical
#my $values = np->arange(40)->reshape([1, 10, 4]);
#my $valid_lens = np->array([2, 6]);
#my $attention = AdditiveAttention->new(num_hiddens=>8, dropout=>0.1);


numperl=ARRAY(0xc9a8268)

In [236]:
package Seq2SeqAttentionDecoder{
    use strict;
  use warnings;
  use constant np => 'numperl';
  use Data::Dump qw(dump);
  use AI::MXNet::Gluon qw(gluon);
  use AI::MXNet::Gluon::NN qw(nn);
  use base ("AttentionDecoder");
  
    sub new{
    my ($class, %args) = (shift, @_);
    my $self = $class->SUPER::new();

    # Embedding layer
    $self->{'attention'} = AdditiveAttention->new($args{'num_hiddens'},$args{'dropout'});
    $self->register_child($self->{'attention'});
    $self->{'embedding'} = gluon->nn->Embedding($args{'vocab_size'}, $args{'embed_size'});
    $self->register_child($self->{'embedding'});
    $self->{'rnn'} = gluon->rnn->GRU(hidden_size => $args{'num_hiddens'}, num_layers => $args{'num_layers'}, dropout => $args{'dropout'} // 0);
    $self->register_child($self->{'rnn'});
    $self->{'dense'} = gluon->nn->Dense($args{'vocab_size'}, flatten => 0);
    $self->register_child($self->{'dense'});
    
    
    return bless($self, $class);
  }
  
   sub init_state{
    my ($self, $enc_outputs, $enc_valid_lens) = (shift, @_, @_);
    my($outputs, $hidden_state)= $enc_outputs;
    return ($outputs->swapaxes(0, 1),  $hidden_state, $enc_valid_lens);
  }
  sub forward{
    my ($self, $X, $state) = (shift, @_, @_);
    my($enc_outputs, $hidden_state, $enc_valid_lens) = $state;
     $X = $self->{'embedding'}->forward($X);
     $X = $X->swapaxes(0, 1);
     my $outputs=[];
     $self->{'attention_weights1'}=[];
   
     for my $x(@$X){
     
        my $query = np->expand_dims($hidden_state->[0][-1], axis=>1); 
        my $context = $self->{'attention'}->($query, $enc_outputs, $enc_outputs, $enc_valid_lens);
        $x = np->concatenate(($context, np->expand_dims($x, axis=>1)), axis=>-1);
        my($out, $hidden_state) = $self->{'rnn'}->forward($x->swapaxes(0, 1), $hidden_state);
        push(@$outputs, $out);
        push(@{$self->attention_weights1}, $self->attention_weights($self->{'attention'}));
     }
    $outputs = $self->{'dense'}->(np->concatenate($outputs, axis=>0));
    return $outputs->swapaxes(0,1), [$enc_outputs, $hidden_state, $enc_valid_lens];
  
  }
  sub attention_weights{
      my $self=@_;
      return $self->attention_weights1;
  }
  
  
}

Warning: Subroutine new redefined at reply input line 10.

Subroutine init_state redefined at reply input line 28.

Subroutine forward redefined at reply input line 33.

Subroutine attention_weights redefined at reply input line 54.


In [237]:
my $encoder = Seq2SeqEncoder->new(vocab_size => 10, embed_size => 8, num_hiddens => 16, num_layers => 2);
$encoder->initialize();

my $decoder = Seq2SeqAttentionDecoder->new(vocab_size=>10, embed_size=>8, num_hiddens=>16, num_layers=>2);
$decoder->initialize();
my $X = mx->nd->zeros([4, 7]);
my $state = $decoder->init_state($encoder->($X), None);
$state = $decoder->($X, $state);
my $output= $decoder->($X, $state);
$output->shape;
$state->len;
$state->[0]->shape;
$state->[1]->len;
$state->[1][0]->shape;

Error: Attribute (units) does not pass the type constraint because: Validation failed for 'Int' with value undef at /usr/local/lib/perl5/site_perl/5.32.1/x86_64-linux/Mouse/Util.pm line 395.
	Mouse::Util::throw_error(Mouse::Meta::Attribute=HASH(0x8aa1000), "Attribute (units) does not pass the type constraint because: "..., "data", undef, "depth", -1) called at reply input line 13
	AdditiveAttention::new("AdditiveAttention", 16, undef) called at reply input line 15
	Seq2SeqAttentionDecoder::new("Seq2SeqAttentionDecoder", "vocab_size", 10, "embed_size", 8, "num_hiddens", 16, "num_layers", 2) called at reply input line 4
	Eval::Closure::Sandbox_1509::__ANON__() called at /usr/local/lib/perl5/site_perl/5.32.1/Reply/Plugin/Defaults.pm line 71
	Reply::Plugin::Defaults::execute(Reply::Plugin::Defaults=HASH(0x4c2db40), CODE(0xcd8f258), CODE(0xc93c190)) called at /usr/local/lib/perl5/site_perl/5.32.1/Reply.pm line 217
	Reply::_wrapped_plugin(Reply=HASH(0x4c141d0), ARRAY(0xc9f5f68), "execute", C

In [197]:
# @Save
sub try_gpu{
  my $i = 0;
  #Return gpu(i) if exists, otherwise return cpu().
  #return mx->context->num_gpus() >= $i + 1 ? mx->gpu($i) : mx->cpu($i);
  return mx->cpu($i);
}

Warning: Subroutine try_gpu redefined at reply input line 2.


In [198]:
#@save
sub train_seq2seq{
  my ($net, $data_iter, $lr, $num_epochs, $tgt_vocab, $device) = @_;
  #Train a model for sequence to sequence.

  $net->initialize(mx->init->Xavier(), force_reinit => 1, ctx => $device);
  my $trainer = AI::MXNet::Gluon::Trainer(params => $net->collect_params(), 
       optimizer => 'adam', optimizer_params  => {"learning_rate" => $lr});

  my $loss = MaskedSoftmaxCELoss();
  my $animator = Animator->new(xlabel => 'Epoch', ylabel => 'Loss', xlim => [10, $num_epochs]);
  my ($timer, $metric, $X, $X_valid_len, $Y, $Y_valid_len, $bos, $dec_input, $Y_hat, $l, $num_tokens);

  for my $epoch (1 .. $num_epochs){
    $timer = Timer->new();
    $metric = Accumulator->new(2);  # Sum of training loss, no. of tokens
    
    while(my $batch = <$data_iter>){
      $X           = nd->array($batch->[0]->slice('X', [0, -2])->asarray);
      $X_valid_len = nd->array($batch->[0]->slice('X', [-1])->reshape([-1])->asarray);
      $Y           = nd->array($batch->[1]->slice('X', [0, -2])->asarray);
      $Y_valid_len = nd->array($batch->[1]->slice('X', [-1])->reshape([-1])->asarray);

      $bos = mx->nd->array(
          [($tgt_vocab->getitem(['<bos>'])) x $Y->shape->[0]], ctx => $device)->reshape([-1, 1]);
      my $y_slice = nd->array($Y->slice('X', [0, -2])->asarray);
      $dec_input = nd->concatenate([$bos, $y_slice], axis => 1);
      
      autograd->record(
        sub {
          ($Y_hat, undef) = $net->($X, $dec_input, $X_valid_len);
        }
      );
      
      $l = $loss->($Y_hat, $Y, $Y_valid_len);
      $l->backward;
      grad_clipping($net, 1);
      $num_tokens = Y_valid_len->sum();
      $trainer->step($num_tokens);
      $metric->add($l->sum(), $num_tokens);
    }
    
    if (($epoch + 1) % 10 == 0){
      $animator->add($epoch + 1, ($metric->getitem(0) / $metric->getitem(1)));
    }
  }
  
  print sprintf("Loss %.3f %.1f tokens/sec on %s.\n", 
                 ($metric->getitem(0) / $metric->getitem(1)), 
                 ($metric->getitem(1) / $timer->stop()), $device);
}

Warning: Subroutine train_seq2seq redefined at reply input line 2.


In [132]:
my ($embed_size, $num_hiddens, $num_layers, $dropout )= (32, 32, 2, 0.1);
my ($batch_size, $num_steps) = (64, 10);
my ($lr, $num_epochs, $device)= (0.005, 250, try_gpu());
my ($train_iter, $src_vocab, $tgt_vocab) = d2l->load_data_nmt(batch_size => $batch_size, num_steps => $num_steps);
my $encoder = Seq2SeqEncoder->new(vocab_size => $src_vocab->len, embed_size => $embed_size, num_hiddens => $num_hiddens, num_layers => $num_layers, dropout => $dropout);
$encoder->initialize();
my $decoder = Seq2SeqAttentionDecoder->new(vocab_size => $tgt_vocab->len, embed_size => $embed_size, num_hiddens => $num_hiddens, num_layers => $num_layers, dropout => $dropout);
$decoder->initialize();

my $net = EncoderDecoder->new($encoder, $decoder);
train_seq2seq->new($net, $train_iter, $lr, $num_epochs, $tgt_vocab, $device);

Error: Attribute (units) does not pass the type constraint because: Validation failed for 'Int' with value undef at /usr/local/lib/perl5/site_perl/5.32.1/x86_64-linux/Mouse/Util.pm line 395.
	Mouse::Util::throw_error(Mouse::Meta::Attribute=HASH(0x8aa1000), "Attribute (units) does not pass the type constraint because: "..., "data", undef, "depth", -1) called at reply input line 13
	AdditiveAttention::new("AdditiveAttention", 32, 0.1) called at reply input line 15
	Seq2SeqAttentionDecoder::new("Seq2SeqAttentionDecoder", "vocab_size", 201, "embed_size", 32, "num_hiddens", 32, "num_layers", 2, "dropout", 0.1) called at reply input line 7
	Eval::Closure::Sandbox_1404::__ANON__() called at /usr/local/lib/perl5/site_perl/5.32.1/Reply/Plugin/Defaults.pm line 71
	Reply::Plugin::Defaults::execute(Reply::Plugin::Defaults=HASH(0x4c2db40), CODE(0xaa71238), CODE(0xc0747d8)) called at /usr/local/lib/perl5/site_perl/5.32.1/Reply.pm line 217
	Reply::_wrapped_plugin(Reply=HASH(0x4c141d0), ARRAY(0xb62735